In [1]:
pip install -U deep-translator
!pip install -U datasets

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/ec/93/454ada0d1b289a0f4a86ac88dbdeab54921becabac45da3da787d136628f/datasets-2.16.1-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow-hotfix from https://files.pythonhosted.org/packages/e4/f4/9ec2222f5f5f8ea04f66f184caafd991a39c8782e31f5b0266f101cb68ca/pyarrow_hotfix-0.6-py3-none-any.whl.metadata
  Obtaining dependency information for fsspec[http]<=2023.10.0,>=2023.1.0 from https://files.pythonhosted.org/packages/e8/f6/3eccfb530aac90ad1301c582da228e4763f19e719ac8200752a4841b0b2d/fsspec-2023.10.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.4 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.12.2
    Uninstalling fsspec-2023.12.2:
      Successfully uninstalled fsspec-2023.12.2
  Attempting uninstall:

In [3]:
from deep_translator import GoogleTranslator
from_lang = 'en'
to_lang = 'kn'

translator = GoogleTranslator(source='auto', target=to_lang)

In [4]:
from datasets import load_dataset

dataset = load_dataset("fka/awesome-chatgpt-prompts", split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
dataset

Dataset({
    features: ['act', 'prompt'],
    num_rows: 153
})

In [6]:
act_len = []
prompt_len = []
both = []
for i in dataset['act']:
    act_len.append(len(i))
for i in dataset['prompt']:
    prompt_len.append(len(i))


In [7]:
len(act_len), len(prompt_len)

(153, 153)

In [8]:
for i in range(len(act_len)):
    both.append(act_len[i] + prompt_len[i])

In [9]:
min(act_len), max(act_len), min(prompt_len), max(prompt_len), min(both), max(both)

(4, 38, 156, 1665, 177, 1691)

In [9]:
chk = ""
for i in dataset['prompt']:
    if len(i) > 500:
        print(i)
        chk = i
        break

I want you to act as an English translator, spelling corrector and improver. I will speak to you in any language and you will detect the language, translate it and answer in the corrected and improved version of my text, in English. I want you to replace my simplified A0-level words and sentences with more beautiful and elegant, upper level English words and sentences. Keep the meaning same, but make them more literary. I want you to only reply the correction, the improvements and nothing else, do not write explanations. My first sentence is "istanbulu cok seviyom burada olmak cok guzel"


In [10]:
chk, len(chk), chk[0:500].rfind(".")

('I want you to act as an English translator, spelling corrector and improver. I will speak to you in any language and you will detect the language, translate it and answer in the corrected and improved version of my text, in English. I want you to replace my simplified A0-level words and sentences with more beautiful and elegant, upper level English words and sentences. Keep the meaning same, but make them more literary. I want you to only reply the correction, the improvements and nothing else, do not write explanations. My first sentence is "istanbulu cok seviyom burada olmak cok guzel"',
 594,
 422)

In [11]:
start = 0
stop = len(chk)
threshold = 500
splits = []
while True:
    if threshold < stop:
        idx = chk[start: threshold].rfind(".") + 1
        splits.append(chk[start:idx])
        start += idx
        threshold += idx
    else:
        splits.append(chk[start:threshold])
        break

In [12]:
splits

['I want you to act as an English translator, spelling corrector and improver. I will speak to you in any language and you will detect the language, translate it and answer in the corrected and improved version of my text, in English. I want you to replace my simplified A0-level words and sentences with more beautiful and elegant, upper level English words and sentences. Keep the meaning same, but make them more literary.',
 ' I want you to only reply the correction, the improvements and nothing else, do not write explanations. My first sentence is "istanbulu cok seviyom burada olmak cok guzel"']

In [14]:
for i in dataset['prompt']:
    if "." not in i and len(i) > 500:
        print(len(i))
        print(i)
        print("-"*200)

502
Calling out for assistance from knowledgeable personnel with experience of arranging flowers professionally to construct beautiful bouquets which possess pleasing fragrances along with aesthetic appeal as well as staying intact for longer duration according to preferences; not just that but also suggest ideas regarding decorative options presenting modern designs while satisfying customer satisfaction at same time! Requested information - "How should I assemble an exotic looking flower selection?"
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


In [18]:
st = """
Calling out for assistance from knowledgeable personnel with experience of arranging flowers professionally to construct beautiful bouquets which possess pleasing fragrances along with aesthetic appeal as well as staying intact for longer duration according to preferences; not just that but also suggest ideas regarding decorative options presenting modern designs while satisfying customer satisfaction at same time! Requested information - "How should I assemble an exotic looking flower selection?"
"""
len(st)

504

In [19]:
st = st.replace(";", "")
st = st.replace("!", "")

In [20]:
len(st)

502

In [23]:
dataset['prompt'][54], len(dataset['prompt'][54])

('Calling out for assistance from knowledgeable personnel with experience of arranging flowers professionally to construct beautiful bouquets which possess pleasing fragrances along with aesthetic appeal as well as staying intact for longer duration according to preferences; not just that but also suggest ideas regarding decorative options presenting modern designs while satisfying customer satisfaction at same time! Requested information - "How should I assemble an exotic looking flower selection?"',
 502)

In [26]:
dataset['prompt'][54] = dataset['prompt'][54].replace(";", "")
dataset['prompt'][54] = dataset['prompt'][54].replace("!", "")

In [30]:
dataset['prompt']

Dataset({
    features: ['act', 'prompt'],
    num_rows: 153
})

In [29]:
dataset['prompt'][54], len(dataset['prompt'][54])

('Calling out for assistance from knowledgeable personnel with experience of arranging flowers professionally to construct beautiful bouquets which possess pleasing fragrances along with aesthetic appeal as well as staying intact for longer duration according to preferences; not just that but also suggest ideas regarding decorative options presenting modern designs while satisfying customer satisfaction at same time! Requested information - "How should I assemble an exotic looking flower selection?"',
 502)

In [35]:
def translate_chunks(example):
    prompt = example['prompt']
    start = 0
    stop = len(prompt)
    threshold = 500
    splits = []
    if "." not in prompt and stop==502:
        prompt = prompt.replace(";", "")
        prompt = prompt.replace("!", "")
        print(len(prompt))
        if len(prompt) > 500:
            return
        kannada_trans = translator.translate(prompt)
    elif stop > 500:
        while True:
            if threshold < stop:
                idx = prompt[start: threshold].rfind(".") + 1
                splits.append(prompt[start:idx])
                start += idx
                threshold += idx
            else:
                splits.append(prompt[start:threshold])
                break

        kannada_trans = ""
        for i in splits:
            translation = translator.translate(i)
            kannada_trans += translation
    else:
        kannada_trans = translator.translate(prompt)
        
    example['kannada_prompt'] = kannada_trans
    return example



In [36]:
dataset = dataset.map(
    translate_chunks
)

Map:   0%|          | 0/153 [00:00<?, ? examples/s]

500


In [37]:
dataset

Dataset({
    features: ['act', 'prompt', 'kannada_prompt'],
    num_rows: 153
})

In [39]:
dataset['kannada_prompt'][0]

'MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE TRANSLATIONS FOR TODAY. NEXT AVAILABLE IN  18 HOURS 37 MINUTES 23 SECONDS VISIT HTTPS://MYMEMORY.TRANSLATED.NET/DOC/USAGELIMITS.PHP TO TRANSLATE MORE'

In [40]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")

In [42]:
DATASET_PATH = "Sharathhebbar24/awesome_chatgpt_prompts_kannada"
dataset.push_to_hub(
    DATASET_PATH,
    token=HF_TOKEN,
)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Sharathhebbar24/awesome_chatgpt_prompts_kannada/commit/e85e243eb56a90e2a868fcc06754a9abfe370d56', commit_message='Upload dataset', commit_description='', oid='e85e243eb56a90e2a868fcc06754a9abfe370d56', pr_url=None, pr_revision=None, pr_num=None)